In [1]:
import boto3
import xmltodict as xml
import tempfile
from pprint import pprint

# Uncomment this line to use in production
# endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'

region_name = 'us-east-1'
endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
bucket='pgdp-3-bucket'
target_bucket='pgdp-mturk-selected'

In [2]:
def aws_clients(aws_access_key_id, aws_secret_access_key):
        
    mturk_client = boto3.client(
    'mturk',
    endpoint_url=endpoint_url,
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key)
    
    print("MTurk Client Created")
    
    s3_client = boto3.client(
    's3',
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key)
    
    print("S3 Client Created")
    
    return mturk_client, s3_client

In [3]:
mturk_client, s3_client = aws_clients()

MTurk Client Created
S3 Client Created


In [4]:
response = s3_client.list_objects_v2(Bucket=bucket)
track_hits=[]
for ele in response['Contents']:
    #print(ele['Key'])
    tag_response = s3_client.get_object_tagging(Bucket=bucket, Key=ele['Key'])
    if not tag_response['TagSet']:
        create_hit_resp = mturk_client.create_hit(
                            MaxAssignments=2,
                            LifetimeInSeconds=72000,
                            AssignmentDurationInSeconds=3600,
                            Reward='0.10',
                            Title='License Plate Identification',
                            Description='License Plate Identification',
                            HITLayoutId='3P8MHVAMIX6K1ZERMHKAGCB2Z59HI5',
                            HITLayoutParameters=[{'Name':'image_url', 'Value':f"https://{bucket}.s3.amazonaws.com/{ele['Key']}"}]
                            )
        s3_client.put_object_tagging(Bucket=bucket, Key=ele['Key'], Tagging={'TagSet':[{'Key':'sent_to_mturk', 'Value':'True'}]})
        track_hits.append({ create_hit_resp['HIT']['HITId']:ele['Key'] })

In [5]:
track_hits

[]

In [23]:
#track_hits=[{'30UZJB2POHQBE808VHR94L8PW8953H': '31.jpg'}]
#r= mturk_client.get_hit(HITId='30UZJB2POHQBE808VHR94L8PW8953H')
r=mturk_client.list_reviewable_hits()
from pprint import pprint
pprint(r['HITs'])

[{'HITId': '30UZJB2POHQBE808VHR94L8PW8953H'},
 {'HITId': '39TX062QX12KGRKI1NA4W4Z8O3IX37'},
 {'HITId': '3DA79LNS599DZABYWD9771E2B7UT3K'},
 {'HITId': '3ZICQFRS31WA6FRQNCTYXQKYBYFZZ3'},
 {'HITId': '3OLZC0DJ8JT725X6MBG9CP4CRA7VI2'},
 {'HITId': '37G6BXQPLQZ3HSZ34TW8LN7CIPLQEU'},
 {'HITId': '3MYASTQBG7PYPML38YLOF5PRIDKQD1'},
 {'HITId': '3TLFH2L6Y92OESFFSDV2OR9A5TUT29'},
 {'HITId': '3KLL7H3EGDF76AMMGTYOCD3XRXSVH9'},
 {'HITId': '371QPA24C22HP7XJ99VOJ8Z5DNGT1S'}]


In [21]:
mturk_client.list_assignments_for_hit(HITId='30UZJB2POHQBE808VHR94L8PW8953H')['Assignments'][0]

{'AssignmentId': '3OJSZ2ATDTAW1BZPNVZ15MGL5VQ75W',
 'WorkerId': 'A3HNG9AAEZKB19',
 'HITId': '30UZJB2POHQBE808VHR94L8PW8953H',
 'AssignmentStatus': 'Submitted',
 'AutoApprovalTime': datetime.datetime(2019, 12, 31, 15, 0, 33, tzinfo=tzlocal()),
 'AcceptTime': datetime.datetime(2019, 12, 1, 15, 0, 9, tzinfo=tzlocal()),
 'SubmitTime': datetime.datetime(2019, 12, 1, 15, 0, 33, tzinfo=tzlocal()),
 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>Multiple.m</QuestionIdentifier><FreeText>false</FreeText></Answer><Answer><QuestionIdentifier>Other.o</QuestionIdentifier><FreeText>false</FreeText></Answer><Answer><QuestionIdentifier>Single.s</QuestionIdentifier><FreeText>true</FreeText></Answer><Answer><QuestionIdentifier>tag1</QuestionIdentifier><FreeText>A193958</FreeText></Answer></QuestionFormAnswers>'}

In [24]:
# s3 = boto3.resource('s3')
# tgt_bucket = s3.Bucket('pgdp-mturk-selected')

for ele in mturk_client.list_reviewable_hits()['HITs']:
    #hit=list(ele.keys())[0]
    hit = ele['HITId']
    m_assgn = mturk_client.list_assignments_for_hit(HITId=hit)['Assignments']
    ans1=ans2=None
    cnt=1
    for assgn in m_assgn:
        xml = assgn['Answer']
        ans = xmltodict.parse(x)['QuestionFormAnswers']['Answer']
        print(ans)
        flag = False
        for i in ans:
            print(i['QuestionIdentifier'] +' '+ i['FreeText'])
            if i['QuestionIdentifier'] == 'Single.s' and i['FreeText'] == 'true':
                flag = True
                continue

            if flag and i['QuestionIdentifier'] == 'tag1':
                if cnt == 1:
                    ans1 = i['FreeText']
                    cnt += 1
                else:
                    ans2 = i['FreeText']
                print("Update Databse for HIT", ans1 , ans2)
    
        if ans1 == ans2:
            mturk_client.approve_assignment(AssignmentId=assgn['AssignmentId'])
            #Update Status with labeled status
        elif ans1!=None and ans2!=None:
            mturk_client.reject_assignment(AssignmentId=assgn['AssignmentId'])
        
#     if class_ == "Single":
#         tmp = tempfile.NamedTemporaryFile()
#         with open('tmp.jpg', 'wb') as data:
#             s3_client.download_fileobj(bucket, ele[hit], data)
        
#         with open('tmp.jpg', 'rb') as data:
#             s3_client.upload_fileobj(data, target_bucket, ele[hit])

[OrderedDict([('QuestionIdentifier', 'Multiple.m'), ('FreeText', 'false')]), OrderedDict([('QuestionIdentifier', 'Other.o'), ('FreeText', 'false')]), OrderedDict([('QuestionIdentifier', 'Single.s'), ('FreeText', 'true')]), OrderedDict([('QuestionIdentifier', 'tag1'), ('FreeText', 'A193958')])]
Multiple.m false
Other.o false
Single.s true
tag1 A193958
Update Databse for HIT A193958 None
[OrderedDict([('QuestionIdentifier', 'Multiple.m'), ('FreeText', 'false')]), OrderedDict([('QuestionIdentifier', 'Other.o'), ('FreeText', 'false')]), OrderedDict([('QuestionIdentifier', 'Single.s'), ('FreeText', 'true')]), OrderedDict([('QuestionIdentifier', 'tag1'), ('FreeText', 'A193958')])]
Multiple.m false
Other.o false
Single.s true
tag1 A193958
Update Databse for HIT A193958 A193958
[OrderedDict([('QuestionIdentifier', 'Multiple.m'), ('FreeText', 'false')]), OrderedDict([('QuestionIdentifier', 'Other.o'), ('FreeText', 'false')]), OrderedDict([('QuestionIdentifier', 'Single.s'), ('FreeText', 'true')

In [ ]:
import requests
import matplotlib.image as mpimg
#import tempfile

API_TOKEN='f0db6826198504391f83f27119219ae160757112'
target_bucket='pgdp-mturk-selected'

# resp=s3_client.list_objects_v2(Bucket=target_bucket)
# for ele in resp['Contents']:
#     print(ele['Key'])

with open('tmp.jpg', 'wb') as data:
    s3_client.download_fileobj(target_bucket, '11.jpg', data)
    
with open('tmp.jpg', 'rb') as data:
    response = requests.post(
            'https://api.platerecognizer.com/v1/plate-reader/',
            files=dict(upload=data),
            headers={'Authorization': f'Token {API_TOKEN}'}
    )
    

In [17]:
pprint(response.json())

{'camera_id': None,
 'filename': '20_37_btmp.jpg',
 'processing_time': 174.858,
 'results': [{'box': {'xmax': 705, 'xmin': 18, 'ymax': 389, 'ymin': 73},
              'candidates': [{'plate': '2rr168', 'score': 0.9},
                             {'plate': '2rri68', 'score': 0.759},
                             {'plate': '2rr16b', 'score': 0.758},
                             {'plate': '2rri6b', 'score': 0.617}],
              'dscore': 0.682,
              'plate': '2rr168',
              'score': 0.9,
              'vehicle': {'type': 'Unknown'}}],
 'timestamp': '2019-11-22T20:37:48.242635Z',
 'version': 1}
